<a href="https://colab.research.google.com/github/JoaoVM11/Projeto-mecanismo-de-consulta-para-landing-pages/blob/main/Projeto_mecanismo_de_consulta_para_landing_pages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [2]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [4]:
#Exemplo de embedding
title = "Tabela de Serviços Empresa_Fictícia"
sample_text = ("Conteudo: O valor da Consultoria é 30.500,00 por mês, sua duração de 12 meses"
    "\n"
    "Conheça nossos serviços, organize sua estrutra e alavanque suas vendas.")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.059551995, -0.0063764676, 0.0062257, 0.0034201078, 0.055705283, 0.013996, -0.002932961, -0.023914397, 0.02668843, 0.01984695, -0.05138093, -0.023948276, 0.025648165, -0.00613173, -0.026942736, -0.016943047, -0.007781463, 0.0052388753, 0.0039179055, 0.02255945, 0.0074475915, 0.025969617, -0.03987295, -0.03330559, -0.008233174, -0.0038512975, 0.004577, -0.06472442, -0.040192734, 0.012853811, -0.0725131, 0.05142332, -0.0007573846, -0.007015139, 0.014238873, -0.052108105, -0.021755518, 0.045042902, -0.009770007, -0.026089346, -0.008609272, -0.023415944, -0.020239625, 0.037927072, -0.0115253525, -0.013167165, -0.014615425, 0.0011315362, 0.010331684, -0.07927557, 0.018232962, -0.024149612, 0.08767602, -0.0110095665, 0.03409739, -0.04605975, 0.07259311, -0.050476, -0.026555413, 0.0012316745, 0.013812553, -0.00914149, 0.04315247, 0.025077894, -0.018546244, -0.043314494, -0.037580993, 0.0006870041, 0.052769084, 0.003309352, -0.030674666, -0.028352551, 0.014955477, -0.010478469,

In [6]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Titulo": "Treinamento Vendedores Internos",
    "Conteudo": "10.000,00 por turma"
}

DOCUMENT2 = {
    "Titulo": "Treinamento Vendedores Externos",
    "Conteudo": "12.000,00 por turma"
}

DOCUMENT3 = {
    "Titulo": "Consultoria",
    "Conteudo": "30.000,00 por mês, sua duração é de 12 meses"
}

DOCUMENT4 = {
    "Titulo": "Treinamento para Líderes",
    "Conteudo": "15.000,00 por turma"
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4]

In [7]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Treinamento Vendedores Internos,"10.000,00 por turma"
1,Treinamento Vendedores Externos,"12.000,00 por turma"
2,Consultoria,"30.000,00 por mês, sua duração é de 12 meses"
3,Treinamento para Líderes,"15.000,00 por turma"


In [8]:
model = "models/embedding-001"

In [9]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [10]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Treinamento Vendedores Internos,"10.000,00 por turma","[0.031667583, -0.06801705, -0.017075907, -0.01..."
1,Treinamento Vendedores Externos,"12.000,00 por turma","[0.037922647, -0.062519915, -0.017862182, -0.0..."
2,Consultoria,"30.000,00 por mês, sua duração é de 12 meses","[0.050689775, -0.014774876, -0.0009846197, -0...."
3,Treinamento para Líderes,"15.000,00 por turma","[0.04167994, -0.06139321, -0.015102574, -0.019..."


In [11]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [12]:
consulta = "Qual é o valor da Consultoria?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

30.000,00 por mês, sua duração é de 12 meses


In [13]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [14]:
#Mesclando a resposta vinda pelo documento Embeddings com a IA Generativa para melhorar a resposta de acordo com o perfil do cliente
prompt = f"Reescreva essa informação de uma forma que, antes de informar o preço, incentive o cliente a adquirir a Consultoria. Lembrando que esse cliente terá um perfil corporativo, sendo um CEO, Diretor Comercial, Gerente, etc. Não esqueça de informar o preço e a duração do projeto no final. {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Eleve sua Empresa a Novos Patamares com Nossa Consultoria Estratégica**

Como líder corporativo, você enfrenta desafios complexos que exigem soluções inovadoras. Nossa Consultoria Estratégica foi projetada para fornecer a orientação e o suporte necessários para impulsionar o crescimento e o sucesso de sua organização.

Nossa equipe de especialistas trabalhará em estreita colaboração com você para:

* Analisar seu cenário competitivo e identificar oportunidades de mercado
* Desenvolver estratégias personalizadas para atingir seus objetivos de negócios
* Implementar soluções práticas que gerem resultados tangíveis
* Monitorar e ajustar continuamente seu plano para garantir o sucesso a longo prazo

Ao investir em nossa Consultoria Estratégica, você terá acesso a:

* Insights valiosos e análises aprofundadas
* Estratégias personalizadas e planos de ação
* Suporte contínuo e orientação de especialistas
* Uma vantagem competitiva no mercado

**Invista em seu Futuro: Adquira Nossa Consultor